# Modelos de datos en el dataset
 - Unique : ID único de 16 caracteres.
 - category : se debe proporcionar los valores categoricos
 - foreign : se debe espesificar el dataset  y  la columna dataset.colum
 - date: fecha maxima:
 - numeric: tipo de dato numerico

**random**: Esto significa que el dataset tendrá como número máximo de
registros la cantidad de valores en la columna con el mayor número de
valores.



In [16]:
import pandas as pd
import numpy as np
import random
import uuid
from scipy.stats import truncnorm
from datetime import datetime

In [2]:
d1 = {
"ds": "dataset1",
"columns": [
{
"name": "area",
"type": "category",
"values": ["TI", "FIN", "HR"]
},
{
"name": "id",
"type": "unique"
}
],
"random": True,
"random_rows":100
}


d2 = {
"ds": "dataset2",
"columns": [
{
"name": "id",
"type": "unique"
},
{
"name": "area",
"type": "foreign",
"values": "dataset1.id"
},
{
"name": "subarea",
"type": "category",
"values": ["SA1", "SA2", "SA3", "SA4"]
}
],
"random": False
}
d3 = {
"ds": "dataset3",
"columns": [
{
"name": "id",
"type": "unique"
},
{
"name": "subarea",
"type": "foreign",
"values": "dataset2.id"
},
{
"name": "income",
"type": "numeric",
"values": {"min": 10, "max": 20}
},
{
"name": "goal",
"type": "numeric",
"values": {"min": 10, "max": 100, "std": 0.5,
"mean": 110/2}
}
],
"random": True,
"random_rows": 1000
}
d4 = {
"ds": "dataset",
"columns": [
{
"name": "Fecha",
"type": "date",
"values": {
"min": "2024-01-01",
"max": "2024-02-28"
}
    
},
    
{
"name": "area",
"type": "category",
"values": ["TI", "FIN", "HR"]
}
],
    
"random": False
}



# Funciones 

In [4]:
#Esta función encuentra el tamaño máximo basado en la configuración proporcionada..

def findMax(config):
    size=0
    
    for column in config["columns"] :
        if column["type"] in ["date","numeric","category"]:
            if config["random"]:
                    size=config["random_rows"]
            else: 
                
                if column["type"]=="date":
                    
                    dateMin=datetime.strptime(column["values"]["min"],'%Y-%m-%d')
                    dateMax=datetime.strptime(column["values"]["max"],'%Y-%m-%d')
                    days=(dateMax-dateMin).days
                    if(days>size):
                        
                        size=days
                elif column["type"]=="category" :
                    
                    if len(column["values"])>size:
                        
                        size=len(column["values"])
                else:
                    if column["values"]["max"]!=None and column["values"]["max"]>size:
                        
                        size=column["values"]["max"]
                
                    
                
    return size
    
# Esta funcion genera n cantidad de ID de 16 caracteres
def generateID(n):
    list_id=[]
    for i in range(n):
        unique_id = uuid.uuid4().hex
        unique_id[16:]
        list_id.append(unique_id)
    return list_id
                
# Esta funcion genera n cantidad de fechas aleatorias en un rango determinado
def get_random_dates( min , max , n ):
    items = pd.date_range( start = min , end = max , freq = 'D' )
    return random.choices( items , k=n )                
                
            
# Esta funcion genera, una lista de tamanio n, dada una lista predeterminada     
def get_random_category(items,n):
    return random.choices( items , k=n )
        
# Esta funcion genera, n cantidad de valores numericos que siguen una funcion normal Truncada
  
def generate_truncated_normal_data(mean, std, min_val, max_val, size):
    # Calcular los parámetros de la distribución normal truncada
    a, b = (min_val - mean) / std, (max_val - mean) / std
    data = truncnorm(a, b, loc=mean, scale=std).rvs(size)
    return data  
# Esta funcion genera, n cantidad de elementos que siguen la distribucion normal

def simpleNormal(min,max,n):
    list=[]
    mean=(min+max)/2
    std=(max-min)/6
    return np.random.normal(mean,std,n)


# Esta funcion, determina si existe un dataset y una determinada columna en el dataset
# Retorna la posicion del dataset en el config_list en caso de que exista
# Si no existe devuele un error

def searchDataset(config_list,name,column_name):
    for i in range(len(config_list)):
        if config_list[i]["ds"]==name:
            columns=[]
            for column in config_list[i]["columns"]:
                
                if column["name"]==column_name:
                    return i
            mensaje=f"No existe la columna \'{column_name}\' en el dataset  \'{name}\' las columnas que podria utilizar son : \n"+ str([ c["name"] for c in config_list[i]["columns"] ] )
            raise ValueError(mensaje)
    raise ValueError("No existe el \' dataset \' "+name)


In [6]:
#Esta funcion permite, construir un dataFrame  a partir de los archivos de configuracion
# recibe la lista de configuracion, la lista para los dataframe, la configuracion actual, y el tamaño del mismo
def build_dataframe(config_list,dataFrame_list,config):
    dataElement={}
    
    
    size=findMax(config)
    
    for column in config["columns"]: 
        if column["type"] in ["unique","category","foreign","date","numeric"]:

            if column["type"]=="foreign" :
                
                names=column["values"].split(".")
                
                index=searchDataset(config_list,names[0],names[1])

                if(dataFrame_list[index] is None):
                    dataFrame_list[index]=build_dataframe(config_list,dataFrame_list,config_list[index])
           
                dataElement[column["name"]]=random.choices(list(dataFrame_list[index][names[1]]),k=size )
                
            elif column["type"]=="unique" :
                    dataElement[column["name"]]=generateID(size)
                
            elif column["type"]== "date" :
                    dataElement[column["name"]]=get_random_dates(column["values"]["min"],column["values"]["max"],size)
                
            elif column["type"]=="category" :
                    dataElement[column["name"]]=get_random_category(column["values"],size)
                
            elif column["type"]=="numeric":
                if "mean" in column["values"].keys():
                    dataElement[column["name"]]=generate_truncated_normal_data(
                            column["values"]["mean"],column["values"]["std"],column["values"]["min"],column["values"]["max"],size)
                else:
                        dataElement[column["name"]]=simpleNormal(column["values"]["min"],column["values"]["max"],size)
                        
    return pd.DataFrame(dataElement)
    

In [12]:
def build_dataframes(config_list):
    dataFrameList=h=np.full(len(config_list),None)

    for i in range(len(config_list)):
        
        if dataFrameList[i] is None :
            dataFrameList[i]=build_dataframe(config_list,dataFrameList,config_list[i])
            
    return dataFrameList


    

In [34]:
config_list=[d1,d2,d3,d4]

dataFrame_list=build_dataframes(config_list)



In [36]:
dataFrame_list[2]

,id,subarea,income,goal
0,d05d889942f54e6594349816eee519fe,4ea2f4383905495ea7ae4e97c9a03bea,17.454865,54.694167
1,9d3016faf37b4648b40a08560fb97c48,4ea2f4383905495ea7ae4e97c9a03bea,13.724056,55.356244
2,5e0de4f65c914797b20d241d2d84e966,13b131facfa6405dbd6dfb6b11a2dcfa,14.700881,54.880413
3,5ebeec790422416d9f98c13c4dd90a28,e121975bd5b4414ab27d32c82aebe953,16.565075,55.543542
4,a2b26a045a3740c2885fe77d1e781023,4ea2f4383905495ea7ae4e97c9a03bea,16.436759,55.039126
...,...,...,...,...
995,464874eaf6984e62b05396488d081ef8,7af61831d5d44503a61278b759889b07,14.386704,54.363858
996,4c36d3c84c134fbdab6a9041d26e800e,4ea2f4383905495ea7ae4e97c9a03bea,17.323622,55.067268
997,10892fc0fda44cf5bbdb634c968a044f,e121975bd5b4414ab27d32c82aebe953,17.358090,54.679634
998,d836b897737649c5980dbe08052b730f,13b131facfa6405dbd6dfb6b11a2dcfa,17.304669,54.659928
